In [1]:
import os
import numpy as np
import open3d as o3d
import json
import cv2
from plyfile import PlyData
import ply_utils

In [2]:
root_path = '/Users/dragos/Licenta/data/audi'
camera_path = os.path.join(root_path, 'cameras.json')
ply_path = os.path.join(root_path, 'test.ply')
new_ply_path = os.path.join(root_path, 'colored_output.ply')
masks_path = os.path.join(root_path, 'masks')


In [3]:
pcd = o3d.io.read_point_cloud(ply_path)
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

ply = PlyData.read(ply_path)
vertex = ply['vertex'].data
rgb = np.stack([vertex['f_dc_0'], vertex['f_dc_1'], vertex['f_dc_2']], axis=-1)
# rgb = np.clip(rgb, 0, 1)
rgb = rgb.astype(np.float64)
# print(rgb.shape, rgb.min(), rgb.max())
# print(rgb)

with open(camera_path, "r") as f:
    cameras = json.load(f)

In [4]:
filtered_cameras = [camera for camera in cameras if camera['id'] == 0]

filtered_cameras

[{'fx': 296.8810729980469,
  'fy': 296.4210510253906,
  'height': 639,
  'id': 0,
  'img_name': '0001.jpg',
  'position': [-0.6026721000671387, -0.13839955627918243, 0.6677852869033813],
  'rotation': [[-0.42500507831573486, -0.254541277885437, 0.8686653971672058],
   [0.30829429626464844, 0.8615708947181702, 0.40329912304878235],
   [-0.8510730862617493, 0.4392087161540985, -0.2876983880996704]],
  'width': 359}]

In [5]:
# new_colors, z = ply_utils.color_points_by_masks(cameras=filtered_cameras, masks_path=masks_path, points=points, colors=rgb)

# print(new_colors.dtype)

In [6]:
new_colors, z = ply_utils.color_points_by_masks_cl(points=points, colors=rgb, cameras=filtered_cameras, mask_dir=masks_path)

# print(new_colors.shape)

/Users/dragos/Licenta/data/audi/masks/0001.png
90


In [7]:
sortedz = np.sort(z)
#only values > 0
# posz = sortedz[sortedz > 0]
#only keep first 3 decimanls
newposz = np.round(sortedz, 3)
# print(sortedz)
print(newposz[:100])

[0.29  0.293 0.295 0.3   0.301 0.302 0.302 0.303 0.303 0.303 0.303 0.304
 0.304 0.304 0.305 0.305 0.306 0.306 0.307 0.308 0.308 0.309 0.309 0.309
 0.31  0.31  0.31  0.311 0.312 0.312 0.313 0.313 0.314 0.314 0.315 0.316
 0.318 0.318 0.319 0.319 0.319 0.32  0.32  0.321 0.322 0.322 0.323 0.323
 0.325 0.326 0.327 0.327 0.328 0.328 0.328 0.328 0.328 0.329 0.329 0.33
 0.331 0.331 0.333 0.333 0.335 0.336 0.341 0.341 0.341 0.343 0.344 0.344
 0.345 0.346 0.346 0.346 0.347 0.347 0.348 0.349 0.351 0.351 0.352 0.353
 0.354 0.356 0.356 0.358 0.37  0.376]


In [9]:
ply_utils.save_ply(vertex, new_colors, new_ply_path)